<a href="https://www.kaggle.com/code/terryeppler/balanceestimator?scriptVersionId=233325447" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>